In [0]:
%matplotlib inline
# !pip install -U plotly
# import plotly.express as px
from urllib.request import urlopen

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import pandas
import time
import io
import json

#You will need to mount the drive to use the relevant datasets in the shared 'Data' folder
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def load_data(ver=2, t_type='unacast'):
  if ver == 2:
    df_X = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/all_X_ver2.csv')
    del df_X['Unnamed: 0']
    df_Y = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/case_diff_Y_v2.csv')
    del df_Y['Unnamed: 0']
  elif ver == 1:
    df_X = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/all_X.csv')
    del df_X['Unnamed: 0']
    df_Y = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/case_diff_Y.csv')
    del df_Y['Unnamed: 0']

  if t_type == 'unacast':
    df_T = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/unacast_T.csv')
    del df_T['Unnamed: 0']
  elif t_type == 'safegraph':
    df_T = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/safegraph_data.csv')
  df_T = df_T.rename(columns = {'FIPS-Code':'fips', 'Social_Distance_Change':'score'})

  df_X['fips'] = df_X['fips'].astype(float).astype(int)
  df_Y['fips'] = df_Y['fips'].astype(float).astype(int)
  df_T['fips'] = df_T['fips'].astype(float).astype(int)

  return df_X, df_Y, df_T

In [0]:
import math
def construct_dataset(df_X, df_Y, df_T, Y_mode='adj_increase', population_adjustment_factor=1000., num_task=3, t_type='safegraph', \
                      average_y=True):
  """
  Returns dataset as a dict {}
  Key: FIPS
  Value: Tuple (X, Y, T) where X, Y, T are arrays
  """
  data = {}

  score = 'score' if t_type == 'safegraph' else 'unacast_score'
  for idx, row in df_X.iterrows():
    if len(df_Y.loc[df_Y['fips']==row['fips']]) == 0 or len(df_T.loc[df_T['fips']==row['fips']]) == 0 :
      continue
    if math.isnan(df_T.loc[df_T['fips']==row['fips']][score].values[0]):
      continue
    id = row['fips']
    x = row.values[3:]
    relevant_x = []  
    # relevant_x.append(x[0])         # ratio of male
    # relevant_x.append(sum(x[2:13])) # ratio of people under 60
    relevant_x.append(x[21]) # ratio (%) of african american population
    relevant_x.append(x[25]) # ratio of hispanic population
    # for k in x[20:26]:
    #  relevant_x.append(k) # race distribution 
    for i, k in enumerate(x[30:37]):
      if i==3:
        continue
      relevant_x.append(k) # 31: poverty, 32: unemployment, 33: median income, 34: pop density, 35: total test per capita, 36: positive test rate, 37: democrat support rate
    relevant_x.append(float(x[39]) / float(x[26])) # current confirmed case / capita
    relevant_x.append(float(x[40]) / float(x[26])) # current confirmed death / capita

    if Y_mode == 'adj_increase':
      relevant_x.append(x[42]) # per capita increase during March 23-28
    else:
      relevant_x.append(x[41])

    y1_key = 'Y1_' + Y_mode
    y1 = df_Y.loc[df_Y['fips']==id][y1_key].values[0] / 7.0

    y2_key = 'Y2_' + Y_mode
    y2 = df_Y.loc[df_Y['fips']==id][y2_key].values[0] / 7.0

    y3_key = 'Y3_' + Y_mode
    y3 = df_Y.loc[df_Y['fips']==id][y3_key].values[0] / 7.0

    # average daily case within 5, 10, 15 days
    if average_y and Y_mode == 'adj_increase':
      y1 = y1
      y2 = (y1+y2) / 2.0
      y3 = (y3 + 2.0*y2) / 3.0

    if Y_mode == 'adj_increase':
      y1 *= population_adjustment_factor
      y2 *= population_adjustment_factor
      y3 *= population_adjustment_factor

    y = [y1, y2, y3] if num_task == 3 else ([y1, y2] if num_task == 2 else [y1])

    t = df_T.loc[df_T['fips']==id][score].values[0]
    # t = 1 if t > 4 else 0 # binarize T (A, B --> 1, C, D, F --> 0)

    id_key = "{:05}".format(id)
    data[id_key] = (relevant_x, y, t)
  
  return data


In [0]:
df_X, df_Y, df_T = load_data(ver=2, t_type='unacast')
data = construct_dataset(df_X, df_Y, df_T, Y_mode='adj_increase', num_task=3, t_type='unacast')

print(len(data))
print(data['25025'][0]) # Suffolk County, Massachusetts
print(data['25025'][1]) # Suffolk County, Massachusetts
print(data['25025'][2]) # Suffolk County, Massachusetts

In [0]:
# Split Train, Val, Test
import random
# test = {'0': 0, '1': 1, '2':2, '3':3}
# keys = list(test.keys())
# shuffledKeys = random.sample(keys, 4)
# print(shuffledKeys)

def generate_split(keys, val_size=0.1, test_size=0.2):
  if val_size + test_size >= 1.0:
    raise ValueError("Sum of the validation size and the test size should be less than 1")
  shuffled_keys = random.sample(keys, len(keys))

  total_len = len(keys)
  train_keys = shuffled_keys[:int(total_len*(1-val_size-test_size))]
  val_keys = shuffled_keys[int(total_len*(1-val_size-test_size)):int(total_len*(1-test_size))]
  test_keys = shuffled_keys[int(total_len*(1-test_size)):]

  return train_keys, val_keys, test_keys

train_keys, val_keys, test_keys = generate_split(list(data.keys()))
print(len(train_keys), len(val_keys), len(test_keys))

In [0]:
def construct_split_dataset(data, train_keys, val_keys, test_keys, censor_by='all'):
  XT_train = []
  XT_val = []
  XT_test = []

  Y_train = []
  Y_val= []
  Y_test = []

  final_train_keys = []
  for key in train_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    x = np.asarray(x)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue
    
    feature_interact = t * x
    feature = np.insert(x, x.shape[0], t)
    feature = np.concatenate((feature, feature_interact))
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0 ):
      continue

    XT_train.append(feature)
    Y_train.append(y)
    final_train_keys.append(key)

  final_val_keys = []
  for key in val_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    x = np.asarray(x)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue

    feature_interact = t * x
    feature = np.insert(x, x.shape[0], t)
    feature = np.concatenate((feature, feature_interact))
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
      continue
    XT_val.append(feature)
    Y_val.append(y)
    final_val_keys.append(key)

  final_test_keys = []
  for key in test_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    x = np.asarray(x)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue

    feature_interact = t * x
    feature = np.insert(x, x.shape[0], t)
    feature = np.concatenate((feature, feature_interact))
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
      continue
    XT_test.append(feature)
    Y_test.append(y)
    final_test_keys.append(key)

  XT_train = np.asarray(XT_train)
  XT_val = np.asarray(XT_val)
  XT_test = np.asarray(XT_test)
  Y_train = np.asarray(Y_train)
  Y_val = np.asarray(Y_val)
  Y_test = np.asarray(Y_test)
  return XT_train, XT_val, XT_test, Y_train, Y_val, Y_test, final_train_keys, final_val_keys, final_test_keys

In [0]:
# Train the MultiTaskLasso
from sklearn import linear_model, metrics, multioutput, neighbors

def train_model(XT_train, Y_train, XT_val, Y_val, scaler, y_scaler, alpha, l1_ratio, y_scale=False, model_type='linear', n_neighbors=20):
  if model_type == 'linear':
    # model = linear_model.MultiTaskLasso(alpha=alpha, max_iter=100000)
    model = linear_model.MultiTaskElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    # model = multioutput.RegressorChain(linear_model.Ridge(alpha=alpha, max_iter=100000))
  elif model_type == 'knn':
    basemodel = neighbors.KNeighborsRegressor(n_neighbors=n_neighbors, weights='distance')
    model = multioutput.RegressorChain(basemodel)
  XT_train_transformed = scaler.transform(XT_train)
  # print(XT_train_transformed.mean(axis=0))
  if y_scale:
    Y_train_ = y_scaler.transform(Y_train)
  else:
    Y_train_ = Y_train

  model.fit(XT_train_transformed, Y_train_)

  pred = model.predict(XT_train_transformed)
  mse = metrics.mean_squared_error(Y_train_, pred)
  print("Mean Squred Error for the Train Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_train_transformed, Y_train_)))

  XT_val_transformed = scaler.transform(XT_val)
  if y_scale:
    Y_val_ = y_scaler.transform(Y_val)
  else:
    Y_val_ = Y_val
  pred = model.predict(XT_val_transformed)
  mse = metrics.mean_squared_error(Y_val_, pred)
  print("Mean Squred Error for the Validation Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_val_transformed, Y_val_)))
  # print("Printing predictions on the validataion set")
  # print("Index , [Ground Truth], [Predicted]")
  # Y_val_ = y_scaler.inverse_transform(Y_val_)
  # pred = y_scaler.inverse_transform(pred)
  # for idx, (y, p) in enumerate(zip(Y_val_, pred)):
  #   print(idx, y, p)

  return model

def test_model(model, XT, Y, scaler, y_scaler, y_scale=True):
  XT_transformed = scaler.transform(XT)
  if y_scale:
    Y_ = y_scaler.transform(Y)
  else:
    Y_ = y
  pred = model.predict(XT_transformed)
  mse = metrics.mean_squared_error(Y_, pred)
  print("Mean Squred Error for the Test Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_transformed, Y_)))
  print("Printing predictions on the validataion set")
  print("Index , [Ground Truth], [Predicted]")
  Y_val_ = y_scaler.inverse_transform(Y_)
  pred = y_scaler.inverse_transform(pred)
  for idx, (y, p) in enumerate(zip(Y_val_, pred)):
    print(idx, y, p)


Including Networking Effect with Simple Markov Assumption

In [0]:
import csv
PATH = '/content/drive/My Drive/COVID19 Data Exploration /Data/county_adjacency.csv'
county_df = pd.read_csv(PATH, header=None)
county_df.head()

In [0]:
county_adjacency = {}
prev_fips = float('nan')
for idx, row in county_df.iterrows():
  fips = row[0]
  neighbor = row[1]
  if math.isnan(fips):
    fips = prev_fips
  else:
    prev_fips = fips
  if int(fips) in county_adjacency.keys():
    county_adjacency[int(fips)].append(int(neighbor))
  else:
    county_adjacency[int(fips)] = [int(neighbor)]

print(county_adjacency[1001])


In [0]:
def check_data_validity(x, y, t, censor_by):
  last_three_x = x[len(x)-3:]
  flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
  if not flag:
    return False
  feature = np.insert(x, len(x), t)
  if np.isnan(feature.sum()):
    return False
  if np.isnan(y.sum()):
    return False
  if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
    return False

  return True

def process_data_with_adjacency(data, keys, adjacency, censor_by='all', neighbor_strength=1.0):
  XT = []
  Y = []
  final_keys = []

  for key in keys:
    x, y, t = data[key]
    y = np.asarray(y)
    x = np.asarray(x)
    valid = check_data_validity(x, y, t, censor_by)
    if valid == False:
      continue
    xt_interact = t * x
    xt = np.insert(x, x.shape[0], t)
    xt = np.concatenate((xt, xt_interact))
    final_keys.append(key)

    adj_xt= []
    if int(key) in adjacency.keys():
      neighbors = adjacency[int(key)]
      for n in neighbors:
        # Exclude itself
        if int(n) == int(key):
          continue
        if "{:05}".format(n) in data.keys():
          a_x, a_y, a_t = data["{:05}".format(n)]
          a_y = np.asarray(a_y)
          a_x = np.asarray(a_x)
          a_valid = check_data_validity(a_x, a_y, a_t, censor_by)
          if a_valid:
            a_xt_interact = a_t * a_x
            a_xt = np.insert(a_x, a_x.shape[0], a_t)
            a_xt = np.concatenate((a_xt, a_xt_interact))
            adj_xt.append(a_xt)
      
    if len(adj_xt) > 0:
      # print(len(adj_xt))
      adj_xt = np.asarray(adj_xt)
      adj_effect = adj_xt.mean(axis=0)
      xt = xt + neighbor_strength * adj_effect
    else:
      xt = (1 + neighbor_strength) * xt
    
    XT.append(xt)
    Y.append(y)

  return XT, Y, final_keys


In [0]:
neighbor_XT_train, neighbor_Y_train, neighbor_final_keys = process_data_with_adjacency(data, train_keys, county_adjacency, \
                                                                                       censor_by='death-only', neighbor_strength=1.0)


In [0]:
neighbor_XT_val, neighbor_Y_val, neighbor_final_val = process_data_with_adjacency(data, val_keys, county_adjacency, censor_by='death-only', neighbor_strength=1.0)
neighbor_XT_test, neighbor_Y_test, neighbor_final_test = process_data_with_adjacency(data, test_keys, county_adjacency, censor_by='death-only', neighbor_strength=1.0)

In [0]:
# Standaradization for XT
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(neighbor_XT_train)
y_scaler = preprocessing.StandardScaler().fit(neighbor_Y_train)

print(scaler.mean_)
print(y_scaler.mean_)

for alpha in [0, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2]:
  for l1 in [0, 0.01, 0.1, 0.2, 0.5]:
    print(alpha, l1)
    model = train_model(neighbor_XT_train, neighbor_Y_train, neighbor_XT_val, neighbor_Y_val, scaler, y_scaler, alpha, l1, True)
# coef = model.coef_
# print(coef)
# for i in range(3):
#   c = coef[i]
#   print(c.shape)
#   print("------------------------------------")
#   print("{}th Task".format(i))
#   # print("Sex (Male ratio): {:.5f}".format(c[0]))
#   # print("Age (<60 ratio) : {:.5f}".format(c[1]))
#   # print("Race - White    : {:.5f}".format(c[0]))
#   print("Race - Black    : {:.5f}".format(c[0]))
#   # print("Race - Native   : {:.5f}".format(c[4]))
#   # print("Race - Pacific  : {:.5f}".format(c[5]))
#   # print("Race - Asian    : {:.5f}".format(c[6]))
#   print("Race - Hispanic : {:.5f}".format(c[1]))
#   print("Bachelor degree : {:.5f}".format(c[2]))
#   print("Poverty         : {:.5f}".format(c[3]))
#   print("Unemployment    : {:.5f}".format(c[4]))
#   # print("Median Income   : {:.5f}".format(c[7]))
#   print("Pop density     : {:.5f}".format(c[5]))
#   print("Test / capita   : {:.5f}".format(c[6]))
#   print("Postive test    : {:.5f}".format(c[7]))
#   # print("Politic - Dem   : {:.5f}".format(c[8]))
#   print("Covid - Case    : {:.5f}".format(c[8]))
#   print("Covid - Death   : {:.5f}".format(c[9]))
#   print("Covid - Increase: {:.5f}".format(c[10]))
#   print("TREATMENT       : {:.5f}".format(c[11]))

In [0]:
model = train_model(neighbor_XT_train, neighbor_Y_train, neighbor_XT_val, neighbor_Y_val, scaler, y_scaler, 0.01, 0.5, True)
test_model(model, neighbor_XT_test, neighbor_Y_test, scaler, y_scaler, y_scale=True)

Analysis: CATE

In [0]:
def cate_measurement(model, scaler, y_scaler, fips, data, df_X, adjacency, neighbor_t=False, censor_by='death-only', neighbor_strength=1.0):
  # Return the expected Y's when T is different (1~9 for Unacast)
  STNAME = df_X.loc[df_X['fips']==int(fips)]['STNAME'].values[0]
  CTYNAME = df_X.loc[df_X['fips']==int(fips)]['CTYNAME'].values[0]
  pop = df_X.loc[df_X['fips']==int(fips)]['total_pop'].values[0]

  print("Fips [{}] {} - {}".format(fips, STNAME, CTYNAME))
  print("Total population: {}".format(pop))
  x, y, t = data[fips]
  y = np.asarray(y)
  x = np.asarray(x)
  print("Original T: {}".format(t))
  print("Original Y1, Y2, Y3: {:.4f}, {:.4f}, {:.4f}".format(y[0]*100, y[1]*100, y[2]*100))

  for tt in range(1, 10):
    feature_interact = tt * x
    feature = np.insert(x, x.shape[0], tt)
    feature = np.concatenate((feature, feature_interact))
    adj_xt= []
    if int(fips) in adjacency.keys():
      neighbors = adjacency[int(fips)]
      for n in neighbors:
        # Exclude itself
        if int(n) == int(fips):
          continue
        if "{:05}".format(n) in data.keys():
          a_x, a_y, a_t = data["{:05}".format(n)]
          if neighbor_t:
            a_t = tt
          a_y = np.asarray(a_y)
          a_x = np.asarray(a_x)
          a_valid = check_data_validity(a_x, a_y, a_t, censor_by)
          if a_valid:
            a_xt_interact = a_t * a_x
            a_xt = np.insert(a_x, a_x.shape[0], a_t)
            a_xt = np.concatenate((a_xt, a_xt_interact))
            adj_xt.append(a_xt)
      
    if len(adj_xt) > 0:
      # print(len(adj_xt))
      adj_xt = np.asarray(adj_xt)
      adj_effect = adj_xt.mean(axis=0)
      feature = feature + neighbor_strength * adj_effect
    else:
      feature = (1+neighbor_strength) * feature

    XT = scaler.transform(feature.reshape((1, -1)))
    pred = model.predict(XT)
    pred = y_scaler.inverse_transform(pred)[0]
    print("When T={}, Y's are: {:.4f}, {:.4f}, {:.4f}".format(tt, pred[0]*100, pred[1]*100, pred[2]*100))

    if tt == 1:
      pred_1 = pred
    if tt == 9:
      pred_9 = pred

  return pred_1, pred_9

In [0]:
cate_1 = 0
cate_2 = 0
cate_3 = 0
for key in neighbor_final_test:
  pred_1, pred_9 = cate_measurement(model, scaler, y_scaler, key, data, df_X, county_adjacency, neighbor_t=True)
  print("----------------------------------------------------------")
  cate_1 += pred_9[0] - pred_1[0]
  cate_2 += pred_9[1] - pred_1[1]
  cate_3 += pred_9[2] - pred_1[2]

cate_1 /= float(len(neighbor_final_test))
cate_2 /= float(len(neighbor_final_test))
cate_3 /= float(len(neighbor_final_test))

print(cate_1*100, cate_2*100, cate_3*100)

In [0]:
cate_measurement(model, scaler, y_scaler, '25025', data, df_X, county_adjacency, neighbor_t=True)
cate_measurement(model, scaler, y_scaler, '25025', data, df_X, county_adjacency, neighbor_t=False)